In [2]:
import weaviate, os
import weaviate.classes as wvc
from tqdm.auto import tqdm
import embed_anything
from embed_anything import EmbedData
from embed_anything.vectordb import Adapter

/home/akshay/miniconda3/envs/embed/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create Weaviate Adapter

In [3]:
from typing import List


class WeaviateAdapter(Adapter):
    def __init__(self, api_key, url):
        super().__init__(api_key)
        self.client = weaviate.connect_to_weaviate_cloud(
            cluster_url=url, auth_credentials=wvc.init.Auth.api_key(api_key)
        )
        if self.client.is_ready():
            print("Weaviate is ready")

    def create_index(self, index_name: str):
        self.index_name = index_name
        self.collection = self.client.collections.create(
            index_name, vectorizer_config=wvc.config.Configure.Vectorizer.none()
        )
        return self.collection

    def convert(self, embeddings: List[EmbedData]):
        data = []
        for embedding in embeddings:
            property = embedding.metadata
            property["text"] = embedding.text
            data.append(
                wvc.data.DataObject(properties=property, vector=embedding.embedding)
            )
        return data

    def upsert(self, embeddings):
        self.client.collections.get(self.index_name).data.insert_many(embeddings)

    def delete_index(self, index_name: str):
        self.client.collections.delete(index_name)

In [4]:
URL = "your-weaviate-url"
API_KEY = "your-weaviate-api-key"
weaviate_adapter = WeaviateAdapter(API_KEY, URL)

WeaviateStartUpError: Could not connect to Weaviate:Connection to Weaviate failed. Details: .

In [86]:
index_name = "Test_index"
if index_name in weaviate_adapter.client.collections.list_all():
    weaviate_adapter.delete_index(index_name)
weaviate_adapter.create_index("Test_index")

## Embed Documents and Stream to Weaviate

In [92]:
embed_config = embed_anything.EmbedConfig(
    cloud=embed_anything.CloudConfig(provider="OpenAI", chunk_size=256)
)
data = embed_anything.embed_file(
    "test.pdf", embeder="OpenAI", adapter=weaviate_adapter, config=embed_config
)

{"total_tokens": 811, "prompt_tokens": 811}


## Query Search

In [94]:
query_vector = embed_anything.embed_query(["What is self attention"], embeder="OpenAI")[
    0
].embedding

{"prompt_tokens": 4, "total_tokens": 4}


In [95]:
response = weaviate_adapter.collection.query.near_vector(
    near_vector=query_vector,
    limit=2,
    return_metadata=wvc.query.MetadataQuery(certainty=True),
)

In [101]:
import textwrap

for res in response.objects:
    print(textwrap.fill(res.properties["text"], width=120), end="\n\n")

2 Background The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU [16],
ByteNet [18] and ConvS2S [9], all of which use convolutional neural networks as basic building block, computing hidden
representations in parallel for all input and output positions. In these models, the number of operations required to
relate signals from two arbitrary input or output positions grows in the distance between positions, linearly for
ConvS2S and logarithmically for ByteNet. This makes it more difficult to learn dependencies between distant positions
[12]. In the Transformer this is reduced to a constant number of operations, albeit at the cost of reduced effective
resolution due to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as described
in section 3. 2. Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representatio